In [1]:
import time

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

import torch 
import torch.nn as nn
import torch.nn.functional as F

import torchvision.datasets as dsets
import torchvision.transforms as trans
from torch.utils.data import DataLoader

In [2]:
train_set = dsets.MNIST(root='data/mnist',
                        train=True,
                        transform=trans.Compose([trans.Resize(32),
                                                 trans.ToTensor()]), 
                        download=True)
test_set = dsets.MNIST(root='data/mnist',
                       train=False,
                       transform=trans.Compose([trans.Resize(32),
                                               trans.ToTensor()]),
                       download=True)

train_dl = DataLoader(train_set,
                      batch_size=100,
                      num_workers=6)
test_dl = DataLoader(test_set,
                     batch_size=100,
                     num_workers=6)

In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.pool1 = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.pool2 = nn.AvgPool2d(2)
        self.fc1 = nn.Linear(5*5*16,120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
        
    def forward(self,x):
        o = self.conv1(x)
        o = self.pool1(o)
        o = F.relu(o)
        
        o = self.conv2(o)
        o = self.pool2(o)
        o = F.relu(o)
        
        o = o.view(x.size(0),-1)
        o = F.relu(self.fc1(o))
        o = F.relu(self.fc2(o))
        o = F.relu(self.fc3(o))
        
        return o

In [4]:
def train_epoch(model,optimizer,criterion,dataloader,device):
    e_total, n_total = 0,0
    for batch_x, batch_y in dataloader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)
        
        optimizer.zero_grad()
        o = model(batch_x)
        e = criterion(o,batch_y)
        e.backward()
        optimizer.step()
        
        e_total += e.item()
        n_total += 1
    return e_total/n_total

In [5]:
def eval_accuracy(model,dataloader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for batch_x, batch_y in dataloader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            batch_o = model(batch_x)
            pred = batch_o.max(1, keepdim=True)[1] 
            correct += pred.eq(batch_y.view_as(pred)).float().mean().item()
    return correct*100.0/len(dataloader)

In [8]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")

print(device)

cpu


In [9]:
net = LeNet()
net = net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(),lr=0.01,momentum=0.9,nesterov=True)

In [10]:
for epoch in range(10):
    time_start = time.time()
    train_loss = train_epoch(net,optimizer,criterion,train_dl,device)
    print('%d/%d\t %.0f seconds\t train error:%.2e'%(epoch+1,10,time.time()-time_start,train_loss))

1/10	 31 seconds	 train error:1.62e+00
2/10	 29 seconds	 train error:2.06e-01
3/10	 29 seconds	 train error:1.22e-01
4/10	 29 seconds	 train error:8.84e-02
5/10	 29 seconds	 train error:6.91e-02
6/10	 27 seconds	 train error:5.71e-02
7/10	 29 seconds	 train error:4.79e-02
8/10	 27 seconds	 train error:3.98e-02
9/10	 26 seconds	 train error:3.43e-02
10/10	 27 seconds	 train error:2.90e-02


In [11]:
eval_accuracy(net,test_dl)

97.7600005865097